<a href="https://colab.research.google.com/github/manvi1711/Python-learning-Journey-/blob/main/python_astro_rec_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
ASTRO_DB = [
    dict(id=1, name="Luna Stardust",
         specialties=["love", "relationships", "soulmate", "dating", "marriage"],
         rating=4.8, price=3.99,
         bio="Expert in love and relationship guidance.",
         languages=["English", "Hindi"]),
    dict(id=2, name="Marcus Celestial",
         specialties=["career", "money", "business", "finance", "job", "wealth"],
         rating=4.7, price=4.50,
         bio="Career & business astrologer for financial growth."),
    dict(id=3, name="Aria Moonchild",
         specialties=["spirituality", "healing", "chakras", "meditation"],
         rating=4.9, price=5.25,
         bio="Spiritual healer focused on chakra alignment."),
    dict(id=4, name="Victor Cosmos",
         specialties=["family", "parenting", "children", "home", "harmony"],
         rating=4.6, price=3.75,
         bio="Family astrologer for parent–child bonding."),
    dict(id=5, name="Zara Mystique",
         specialties=["health", "wellness", "medical astrology", "recovery"],
         rating=4.5, price=4.25,
         bio="Health & wellness advice through medical astrology."),
    dict(id=6, name="Phoenix Starweaver",
         specialties=["travel", "education", "adventure", "study abroad"],
         rating=4.4, price=4.50,
         bio="Travel + education astrology guidance."),
    dict(id=7, name="Sage Timekeeper",
         specialties=["timing", "decisions", "manifestation", "life purpose"],
         rating=4.8, price=5.75,
         bio="Master of cosmic timing for major decisions."),
    dict(id=8, name="Celeste Oracle",
         specialties=["general guidance", "destiny", "future", "insight"],
         rating=4.7, price=4.00,
         bio="Comprehensive life-path and destiny readings.")
]


In [6]:
class AstrologerRecommender:
    def __init__(self, db):
        self.db = db
        docs = [" ".join(a["specialties"]) + " " + a["bio"] for a in db]
        self.vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
        self.tfidf = self.vectorizer.fit_transform(docs)

    def _score_tfidf(self, query):
        q_vec = self.vectorizer.transform([query])
        return cosine_similarity(q_vec, self.tfidf).flatten()

    def _score_keywords(self, query):
        q_words = set(query.lower().split())
        scores = []
        for a in self.db:
            s_words = set(" ".join(a["specialties"]).lower().split())
            b_words = set(a["bio"].lower().split())
            score = (2*len(q_words & s_words) + len(q_words & b_words)) / (len(q_words) or 1)
            scores.append(score)
        return np.array(scores)

    def get_recommendations(self, query, k=3, method="hybrid"):
        if method == "tfidf":
            sims = self._score_tfidf(query)
        elif method == "keywords":
            sims = self._score_keywords(query)
        elif method == "hybrid":
            sims = 0.4*self._score_tfidf(query) + 0.6*self._score_keywords(query)
        else:
            raise ValueError("method must be tfidf | keywords | hybrid")
        top = sims.argsort()[-k:][::-1]
        results = []
        for idx in top:
            a = self.db[idx]
            results.append(
                dict(name=a["name"],
                     relevance=round(float(sims[idx]), 4),
                     rating=a["rating"],
                     price=a["price"],
                     specialties=a["specialties"])
            )
        return results


In [8]:
if __name__ == "__main__":
    system = AstrologerRecommender(ASTRO_DB)
    demo_queries = [
        "I need relationship advice about my boyfriend",
        "Looking for career guidance and business opportunities",
        "Want spiritual healing and chakra alignment",
        "Need help with family issues and parenting",
        "Health problems and wellness advice needed",
        "Planning to travel abroad and study",
        "When should I make major life decisions?",
        "General life guidance and future predictions"
    ]

    for q in demo_queries:
        print(f"\n🔮  Query: {q}")
        for r in system.get_recommendations(q, k=3, method="hybrid"):
            print(f"   • {r['name']:<17} | score {r['relevance']:<4} | "
                  f"⭐{r['rating']} | ${r['price']}/min | {', '.join(r['specialties'][:3])}…")


🔮  Query: I need relationship advice about my boyfriend
   • Luna Stardust     | score 0.1514 | ⭐4.8 | $3.99/min | love, relationships, soulmate…
   • Zara Mystique     | score 0.1375 | ⭐4.5 | $4.25/min | health, wellness, medical astrology…
   • Sage Timekeeper   | score 0.0  | ⭐4.8 | $5.75/min | timing, decisions, manifestation…

🔮  Query: Looking for career guidance and business opportunities
   • Marcus Celestial  | score 0.8027 | ⭐4.7 | $4.5/min | career, money, business…
   • Celeste Oracle    | score 0.2864 | ⭐4.7 | $4.0/min | general guidance, destiny, future…
   • Luna Stardust     | score 0.1163 | ⭐4.8 | $3.99/min | love, relationships, soulmate…

🔮  Query: Want spiritual healing and chakra alignment
   • Aria Moonchild    | score 0.6169 | ⭐4.9 | $5.25/min | spirituality, healing, chakras…
   • Celeste Oracle    | score 0.1  | ⭐4.7 | $4.0/min | general guidance, destiny, future…
   • Luna Stardust     | score 0.1  | ⭐4.8 | $3.99/min | love, relationships, soulmate…

🔮  Query